##FashionMNIST 데이터

In [1]:

import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.datasets import fashion_mnist

# FashionMNIST 데이터 로드
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# 데이터 확인
print(f"Training data shape: {x_train.shape}, Labels: {y_train.shape}")
print(f"Test data shape: {x_test.shape}, Labels: {y_test.shape}")

ImportError: dlopen(/Users/sungkunjeong/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_unuran/unuran_wrapper.cpython-39-darwin.so, 0x0002): tried: '/Users/sungkunjeong/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_unuran/unuran_wrapper.cpython-39-darwin.so' (code signature in <2EB3EB40-3389-366F-9317-8023ADB6E616> '/Users/sungkunjeong/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_unuran/unuran_wrapper.cpython-39-darwin.so' not valid for use in process: library load disallowed by system policy), '/System/Volumes/Preboot/Cryptexes/OS/Users/sungkunjeong/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_unuran/unuran_wrapper.cpython-39-darwin.so' (no such file), '/Users/sungkunjeong/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_unuran/unuran_wrapper.cpython-39-darwin.so' (no such file)